# Segmenting and Clustering Neighborhoods in the city of Toronto

# PART1

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import requests

#!pip install beautifulsoup4
from bs4 import BeautifulSoup

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install folium
import folium # map rendering library

#!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Downloading Dataset from Wikipedia

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response=requests.get(url)
html = response.text
page = BeautifulSoup(html,'html.parser')
table = page.find_all('table')[0]
df = pd.read_html(str(table))
neighbourhood=pd.DataFrame(df[0])
neighbourhood = neighbourhood.rename(columns={0:'Postal Code',1:'Borough',2:'Neighborhood'})
neighbourhood.drop(index=0,inplace=True)


print(neighbourhood.shape)
neighbourhood.head()

(180, 3)


,Postal Code,Borough,Neighborhood
1,M1A,Not assigned,NaN
2,M2A,Not assigned,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Explore Dataset

#### Remove rows where borough equals "Not assigned"

In [3]:
neighbourhood=neighbourhood[~(neighbourhood.Borough=="Not assigned")]
neighbourhood.shape

(103, 3)

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Since no rows satisfy the condition, no need for asigning borough to neighbourhood

In [4]:
neighbourhood[neighbourhood["Neighborhood"]=="Not assigned"].shape
neighbourhood.shape

(103, 3)

# PART2

#### Use geopy library to get coordinates of Toronto

In [5]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.6534817, -79.3839347.


#### Read downloaded csv from (http://cocl.us/Geospatial_data) for getting geographical coordinates of each postal code

In [6]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")

In [7]:
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Mapping lattidues and longitudes to postal codes in Toronto data

In [8]:
map_latitude = dict(coordinates[['Postal Code', 'Latitude']].values)
neighbourhood["Latitude"]= neighbourhood['Postal Code'].map(map_latitude)


map_longitude = dict(coordinates[['Postal Code', 'Longitude']].values)
neighbourhood["Longitude"]= neighbourhood['Postal Code'].map(map_longitude)

In [9]:
neighbourhood.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.753259,-79.329656
4,M4A,North York,Victoria Village,43.725882,-79.315572
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
